# Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import xgboost as xgb
%matplotlib inline

In [2]:
test = pd.read_csv('data/test.csv')
shop = pd.read_csv('data/shops.csv')
submission = pd.read_csv('data/sample_submission.csv')
sales = pd.read_csv('data/sales_train.csv')
items = pd.read_csv('data/items.csv')
item_category = pd.read_csv('data/item_categories.csv')

In [3]:
l = list(item_category.item_category_name)
l_cat = l

for ind in range(1,8):
    l_cat[ind] = 'Access'

for ind in range(10,18):
    l_cat[ind] = 'Consoles'

for ind in range(18,25):
    l_cat[ind] = 'Consoles Games'

for ind in range(26,28):
    l_cat[ind] = 'phone games'

for ind in range(28,32):
    l_cat[ind] = 'CD games'

for ind in range(32,37):
    l_cat[ind] = 'Card'

for ind in range(37,43):
    l_cat[ind] = 'Movie'

for ind in range(43,55):
    l_cat[ind] = 'Books'

for ind in range(55,61):
    l_cat[ind] = 'Music'

for ind in range(61,73):
    l_cat[ind] = 'Gifts'

for ind in range(73,79):
    l_cat[ind] = 'Soft'

item_category['cats'] = l_cat
item_category.head()

,item_category_name,item_category_id,cats
0,PC - Гарнитуры/Наушники,0,PC - Гарнитуры/Наушники
1,Аксессуары - PS2,1,Access
2,Аксессуары - PS3,2,Access
3,Аксессуары - PS4,3,Access
4,Аксессуары - PSP,4,Access


In [4]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

#Compute all shops/items combinations for each date block
grid = []
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# filter extrme values in price and count sells
sales = sales[sales.item_price<100000]
sales = sales[sales.item_cnt_day<=900]

# Grouping sales by (date, shop, item), get the:
+ Sum of item_cnt_dat 
+ Mean of item_price_day

In [5]:
sales_dsi = sales.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': 'sum',
                                                                     'item_price': np.mean}).reset_index()
sales_dsi = pd.merge(grid,sales_dsi,on=['date_block_num','shop_id','item_id'],how='left').fillna(0)
sales_dsi = pd.merge(sales_dsi, items, on=['item_id'],how='left')
sales_dsi = pd.merge(sales_dsi, item_category, on=['item_category_id'], how='left')

# Grouping sales by (date and (shop, item, and category)), and get the: 
+ Average of the of item_price
+ Sum and average of the item_cnt_day

In [6]:
for type_id in ['item_id','shop_id','item_category_id']:
    for column_id, agg_func, func_name in [('item_price',np.mean,'avg'),
                                           ('item_cnt_day',np.sum,'sum'),
                                           ('item_cnt_day',np.mean,'avg')]:

        mean_df = sales_dsi.groupby([type_id,'date_block_num']).agg(agg_func).reset_index()[[column_id,type_id,'date_block_num']]
        mean_df.columns = [type_id+'_'+func_name+'_'+column_id, type_id,'date_block_num']
        
        sales_dsi = pd.merge(sales_dsi,mean_df,on=['date_block_num', type_id],how='left')

temp_test = test.copy()
temp_test['date_block_num'] = 34
temp_test.drop('ID', axis=1, inplace=True)

temp_test = temp_test.merge(items, how='left', on='item_id')
temp_test = temp_test.merge(item_category, how='left', on='item_category_id')
temp_test.drop('item_name', axis=1, inplace=True)
sales_dsi.drop('item_name', axis=1, inplace=True)
sales_dsi = pd.concat([sales_dsi,temp_test], axis=0, ignore_index=True)


In [7]:
features_vars  = ['item_id_avg_item_price', 'item_id_sum_item_cnt_day', 'item_id_avg_item_cnt_day',
                  'shop_id_avg_item_price', 'shop_id_sum_item_cnt_day', 'shop_id_avg_item_cnt_day',
                  'item_category_id_avg_item_price', 'item_category_id_sum_item_cnt_day', 'item_category_id_avg_item_cnt_day',
                  'item_cnt_day']

# creates the lag in a way that for date_block_num ith, the number nth lagged variable corresponds to the 
# value of the non lagged variable in the block (i-n), the non lagged variables are droped from the dataframe 
# to consider only past values.
sales_copy = sales_dsi.copy()
sales_copy.date_block_num+=1
sales_copy = sales_copy[['date_block_num','shop_id','item_id'] + features_vars]
sales_copy.columns = ['date_block_num','shop_id','item_id'] + [lag_feat+'_lag_1' for 
                                                               lag_feat in features_vars]
sales_dsi = sales_dsi.merge(sales_copy,on=['date_block_num','shop_id','item_id'] ,how='left')

sales_copy = sales_dsi.copy()
sales_copy.date_block_num+=2
sales_copy = sales_copy[['date_block_num','shop_id','item_id'] + features_vars]
sales_copy.columns = ['date_block_num','shop_id','item_id'] + [lag_feat+'_lag_2' for 
                                                               lag_feat in features_vars]
sales_dsi = sales_dsi.merge(sales_copy,on=['date_block_num','shop_id','item_id'] ,how='left')

In [8]:
# fills na accordingly to cnt or price
for feat in sales_dsi.columns:
    if 'item_cnt' in feat:
        sales_dsi[feat]=sales_dsi[feat].fillna(0)
    elif 'item_price' in feat:
        sales_dsi[feat]=sales_dsi[feat].fillna(sales_dsi[feat].median())

In [9]:
# filter first 40% of dates considering nly recent history will matter to predict future
cols_to_drop = features_vars[:-1] + ['item_price']
sales_dsi = sales_dsi[sales_dsi['date_block_num'] > 12]
X_train = sales_dsi[sales_dsi['date_block_num']<33].drop(cols_to_drop, axis=1)
X_cv =  sales_dsi[sales_dsi['date_block_num']==33].drop(cols_to_drop, axis=1)
X_test = sales_dsi[sales_dsi['date_block_num']==34].drop(cols_to_drop, axis=1)
for X_df in [X_train, X_cv, X_test]:
    X_df['item_cnt_day'] = X_df['item_cnt_day'].clip_upper(40)
    X_df['item_cnt_day'] = X_df['item_cnt_day'].clip_lower(0)

In [10]:
params = {
        'eta': 0.08, #best 0.08
        'max_depth': 7,
        'objective': 'reg:linear',
        'eval_metric': 'rmse',
        'seed': 3,
        'gamma':1,
        'silent': True
    }
cols = [c for c in X_train.columns if c not in ['date_block_num', 'item_cnt_day', 'item_category_name', "cats"]]

x1 = X_train[cols]
y1 = X_train['item_cnt_day']
x2 = X_cv[cols]
y2 = X_cv['item_cnt_day']

# Final features to train the model with

In [11]:
cols

['item_category_id',
 'item_id',
 'shop_id',
 'item_id_avg_item_price_lag_1',
 'item_id_sum_item_cnt_day_lag_1',
 'item_id_avg_item_cnt_day_lag_1',
 'shop_id_avg_item_price_lag_1',
 'shop_id_sum_item_cnt_day_lag_1',
 'shop_id_avg_item_cnt_day_lag_1',
 'item_category_id_avg_item_price_lag_1',
 'item_category_id_sum_item_cnt_day_lag_1',
 'item_category_id_avg_item_cnt_day_lag_1',
 'item_cnt_day_lag_1',
 'item_id_avg_item_price_lag_2',
 'item_id_sum_item_cnt_day_lag_2',
 'item_id_avg_item_cnt_day_lag_2',
 'shop_id_avg_item_price_lag_2',
 'shop_id_sum_item_cnt_day_lag_2',
 'shop_id_avg_item_cnt_day_lag_2',
 'item_category_id_avg_item_price_lag_2',
 'item_category_id_sum_item_cnt_day_lag_2',
 'item_category_id_avg_item_cnt_day_lag_2',
 'item_cnt_day_lag_2']

In [24]:
watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
model = xgb.train(params, xgb.DMatrix(x1, y1), 3500,  watchlist, maximize=False, verbose_eval=50, early_stopping_rounds=50)

pred = model.predict(xgb.DMatrix(X_test[cols]), ntree_limit=model.best_ntree_limit)

test['item_cnt_month'] = pred.clip(0,20)
test.drop(['shop_id', 'item_id'], axis=1, inplace=True)
test.to_csv('submission2lag.csv', index=False)